In [2]:
import pandas as pd
import numpy as np

In [3]:
cars = pd.read_csv("https://raw.githubusercontent.com/juliandnl/redi_ss20/master/cars.csv")

In [4]:
cars.head()

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba


# Part 1 - Transformations

The following exercises are about doing row-wise transformations of string columns. [Docs](https://pandas.pydata.org/docs/user_guide/text.html#) on working with string data in pandas. [Built-in functions](https://pandas.pydata.org/docs/reference/series.html#string-handling) on string columns. Note that the exercises cover different implementation variations.

As a rule of thumb it is better to use dataframe built-in functionality if available for the logic you want to implement. Lower-level abstractions like `apply` can help if the functionality is not available.

In [5]:
#Exercise 1. Create a new column `has_power` that indicates (with booleans) if the value in the `Variant` column contains the word "Power"
# 1. Use `apply` with a `def` function and anonymous `lambda` function
#do it at home
'''
def power(cars):
    cars['has_power'] = cars['Variant'].str.contains("Power")
    return cars
cars = power(cars)
or with lambda
cars['has_power'] = cars['Variant'].apply(lambda x: "Power" in x)
'''
# 2. Use only dataframe built-in functionality
cars['has_power'] = cars['Variant'].str.contains("Power")
display(cars.head())

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,has_power
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,False
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,False
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,False
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,False
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,True


In [6]:
## Exercise 2
# Create an abbreviation of the `Make` of a car. Create a new column `abbrev_brand` that contains the first 3 letters in uppercase of the value from the `Make` columns. For example, if `Make` is `Chrysler`, `abbrev_brand` should be `CHR`.
# 1. Use `apply` with a regular `def` function and anonymous `lambda` function
'''
def abbrev(cars):
    cars['abbrev_brand'] = cars['Make'].str[:3].str.upper()
    return cars
cars = abbrev(cars)
or with lambda
cars['abbrev_brand'] = cars['Make'].apply(lambda x: x[:3].upper())
'''
# 2. Use only dataframe built-in functionality
cars['abbrev_brand'] = cars['Make'].str[:3].str.upper()
display(cars.head())

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,has_power,abbrev_brand
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,False,VOL
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,False,FOR
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,False,VOL
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,False,FOR
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,True,VOL


# Part 2 - Joins

The following exercises cover different join types. Check the [merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) docs for help on the `how` parameter.

In [7]:
# We create the same dataframe as seen in the class to use for the joins. It is computed based on 'cars' and has following properties
# - Average price **in euros €** of car models per brand (brand = Make). Columns: `Make, Model, avg_price`
# - 3 additional "invented" rows
# - All `Mercedes Benz` cars are removed
Price_Euro = cars['Price'] * 0.0027
cars.insert(6, 'price_euros', Price_Euro)

avg_model_prices = cars.groupby(['Make', 'Model'])['price_euros'].mean().reset_index()
avg_model_prices = avg_model_prices.rename(columns={'price_euros': 'avg_price'})
avg_model_prices = avg_model_prices.loc[avg_model_prices.loc[:, 'Make'] != 'Mercedes Benz', :]

invented_rows = pd.DataFrame(
    data = [('Ford', 'Lo', 158703.340), ('Ford', 'Hi', 324235.670), ('Ford', 'Cheap', 6533.700)],
    columns=['Make', 'Model', 'avg_price']
)
avg_model_prices = pd.concat([invented_rows, avg_model_prices], axis=0, ignore_index=True)

avg_model_prices.shape

(50, 3)

In [8]:
## Exercise 1 Do a RIGHT OUTER JOIN on the dataset and compare the result to the LEFT OUTER JOIN we've done during the class.
right_join = pd.merge(cars, avg_model_prices, on=['Make', 'Model'], how='right')
# 1. How do the `Mercedes Benz` rows compare?
'''display (right_join[right_join['Make'] == 'Mercedes Benz'])
They are not in the right_joindataset, because they were not in a avg_model_prices dataset.'''
# 2. How do the "invented rows" from `avg_model_prices` compare? 
'''They are there, because it was a main dataset'''
# 3. What's the shape of the result? Explain what you think it should be and then check it.
'''So all  columns, but rows without Mercedes Benz
display(cars.head(1),right_join.head(1))
display(cars.shape, avg_model_prices.shape, right_join.shape).  So its 10k-256 rows of MB but +3 new unique values that we invented'''
# 4. How can you achieve the behaviour of the LEFT OUTER JOIN from the class, but doing the RIGHT OUTER JOIN (`merge` with `how='right' parameter`)?
'''right_join = pd.merge(avg_model_prices, cars, on=['Make', 'Model'], how='right')'''

"right_join = pd.merge(avg_model_prices, cars, on=['Make', 'Model'], how='right')"

## Exercise 2
Do a FULL OUTER JOIN and INNER JOIN on the datasets and answer following questions
  1. Check the shape of the result and compare it to the shape of cars/avg_model_prices.
    * What do you observe and how can you explain it? In full outer its 10003*13 because he add 3 unique values that we add and all columns.
  2. Check how `Mercedes Benz` rows look in the result and explain why
  He is in full_outer but not in inner, because into inner_join goes only values that are on both of databases and he is not.
  3. Check how the "invented rows" from `avg_model_prices` look in the result and explain why. Those are only in full_outer_join, because they were eliminated from inner_join (not in both)

In [9]:
inner_join = pd.merge(cars, avg_model_prices,on=['Make', 'Model'])
full_outer_join = pd.merge(cars, avg_model_prices,on=['Make', 'Model'],how='outer')

In [10]:
full_outer_join.tail()
#display (full_outer_join[full_outer_join['Make'] == 'Mercedes Benz'])256 rows

,Make,Model,Year,Variant,Kms,Price,price_euros,Doors,Kind,Location,has_power,abbrev_brand,avg_price
9998,Mercedes Benz,ML,2013.0,3.5 Ml350 4matic Sport B.efficiency,119000.0,49000.0,132.3000,5.0,SUV,Bs.as. G.b.a. Norte,False,MER,NaN
9999,Volkswagen,New Beetle,2011.0,1.8 Turbo Sport,82000.0,299999.0,809.9973,2.0,Cabriolet,Capital Federal,False,VOL,809.9973
10000,Ford,Lo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158703.3400
10001,Ford,Hi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324235.6700
10002,Ford,Cheap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6533.7000


In [11]:
display(cars.shape, avg_model_prices.shape, right_join.shape)

(10000, 12)

(50, 3)

(9747, 13)

In [12]:
display(inner_join)
#display (inner_join[inner_join['Make'] == 'Mercedes Benz']) not there because inner means that it should be in both

,Make,Model,Year,Variant,Kms,Price,price_euros,Doors,Kind,Location,has_power,abbrev_brand,avg_price
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,972.0000,4.0,Sedán,Córdoba,False,VOL,1042.82909
1,Volkswagen,Vento,2013,2.0 Sportline Tsi 200cv,68200,530000,1431.0000,4.0,Sedán,Bs.as. G.b.a. Sur,False,VOL,1042.82909
2,Volkswagen,Vento,2016,2.5 Advance Plus 170cv Tiptronic,59000,500000,1350.0000,4.0,Sedán,Santa Fe,False,VOL,1042.82909
3,Volkswagen,Vento,2014,2.5 Luxury 170cv Tiptronic,65000,470000,1269.0000,4.0,Sedán,Córdoba,False,VOL,1042.82909
4,Volkswagen,Vento,2013,2.5 Luxury 170cv Tiptronic,95000,387500,1046.2500,4.0,Sedán,Bs.as. G.b.a. Norte,False,VOL,1042.82909
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9739,Volkswagen,Virtus,2018,1.6 Msi Trendline,9900,499000,1347.3000,4.0,Sedán,Capital Federal,False,VOL,1412.10000
9740,Volkswagen,Virtus,2018,1.6 Msi Trendline,15000,520000,1404.0000,4.0,Sedán,Bs.as. G.b.a. Norte,False,VOL,1412.10000
9741,Volkswagen,Tiguan Allspace,2018,2.0 Tsi Highline Dsg,16000,45000,121.5000,5.0,SUV,Buenos Aires Interior,False,VOL,1829.25000
9742,Volkswagen,Tiguan Allspace,2018,1.4 Tsi Trendline 150cv Dsg,10000,1310000,3537.0000,5.0,SUV,Bs.as. G.b.a. Oeste,False,VOL,1829.25000


## Exercise 3
Compute the price difference (based on the euros price) of each car, compared to the average price of each brand (`Make`). Do a join as part of the solution. Which type of join do you use? Explain your choice.

In [29]:
avg_make_prices = cars.groupby(['Make'])['price_euros'].mean().reset_index()
avg_make_prices = avg_make_prices.rename(columns={'price_euros': 'avg_make_price'})
cars_with_avg_makeprice = pd.merge(cars, avg_make_prices, on=['Make'], how='inner')
cars_with_avg_makeprice.loc[:, 'price_diff'] = cars_with_avg_makeprice['price_euros'] - cars_with_avg_makeprice['avg_make_price']


In [30]:
cars_with_avg_makeprice

,Make,Model,Year,Variant,Kms,Price,price_euros,Doors,Kind,Location,has_power,abbrev_brand,avg_make_price,price_diff
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,972.000,4.0,Sedán,Córdoba,False,VOL,905.692142,66.307858
1,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,566.946,5.0,Hatchback,Bs.as. G.b.a. Sur,False,VOL,905.692142,-338.746142
2,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,394.200,3.0,Hatchback,Córdoba,True,VOL,905.692142,-511.492142
3,Volkswagen,Amarok,2014,2.0 Cd Tdi 180cv 4x4 Highline C34,115000,790000,2133.000,4.0,Pick-Up,Buenos Aires Interior,False,VOL,905.692142,1227.307858
4,Volkswagen,Amarok,2014,2.0 Cd Tdi 4x4 Highline Pack At C34,79900,685000,1849.500,4.0,Pick-Up,Bs.as. G.b.a. Norte,False,VOL,905.692142,943.807858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Chrysler,PT Cruiser,2011,2.4 Touring,100000,168000,453.600,5.0,Hatchback,Capital Federal,False,CHR,990.483750,-536.883750
9996,Chrysler,PT Cruiser,2011,2.4 Touring,67000,210000,567.000,5.0,Hatchback,Capital Federal,False,CHR,990.483750,-423.483750
9997,Chrysler,Town & Country,2016,3.6 Limited Atx,47000,64000,172.800,5.0,Monovolumen,Bs.as. G.b.a. Oeste,False,CHR,990.483750,-817.683750
9998,Chrysler,PT Cruiser,2011,2.4 Touring,120000,190000,513.000,5.0,Hatchback,Bs.as. G.b.a. Oeste,False,CHR,990.483750,-477.483750


# Bonus Dataset: IMDB movie dataset
We'll look at the movie datasets provided by [IMDb](https://www.imdb.com/). The datasets are described [here](https://developer.imdb.com/non-commercial-datasets/). We'll work with a sample of the `title.basics` dataset (which includes only the top 10,000 most-voted movies) and `title.ratings` datasets.


In [16]:
movies = pd.read_csv("https://raw.githubusercontent.com/obreit/redi/master/imdb_data/top_voted.tsv", sep='\t', header=0)

## Exercise 1
The goal is to get long-running movies (i.e. with `runtimeMinutes` more than 2 hours). Notice that the obvious expression `movies['runtimeMinutes'] > 120` crashes. Try it yourself and try to explain why this is not working (for a hint, see the [details](https://developer.imdb.com/non-commercial-datasets/#imdb-dataset-details) section of the imdb page). Before going to the next hints, try to implement a solution to this problem by yourself.

In [ ]:
movies.info() 
display(movies.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          10000 non-null  object
 1   titleType       10000 non-null  object
 2   primaryTitle    10000 non-null  object
 3   originalTitle   10000 non-null  object
 4   isAdult         10000 non-null  int64 
 5   startYear       10000 non-null  int64 
 6   endYear         10000 non-null  object
 7   runtimeMinutes  10000 non-null  object
 8   genres          10000 non-null  object
dtypes: int64(2), object(7)
memory usage: 703.2+ KB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,Drama
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama"
2,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi"
3,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,Drama
4,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance"


In [ ]:
movies['runtimeMinutes'] = movies['runtimeMinutes'].replace('\\N', pd.NA)

In [ ]:
movies['runtimeMinutes'] = pd.to_numeric(movies['runtimeMinutes'], errors='coerce')

In [ ]:
display(movies[movies['runtimeMinutes'] > 120])

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142.0,Drama
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152.0,"Action,Crime,Drama"
2,tt1375666,movie,Inception,Inception,0,2010,\N,148.0,"Action,Adventure,Sci-Fi"
3,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139.0,Drama
4,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142.0,"Drama,Romance"
...,...,...,...,...,...,...,...,...,...
9989,tt15680228,movie,Bheed,Bheed,0,2023,\N,124.0,"Drama,History"
9990,tt0099426,movie,Bullet in the Head,Dip huet gai tau,0,1990,\N,136.0,"Action,Crime,Drama"
9994,tt5456546,movie,Judwaa 2,Judwaa 2,0,2017,\N,145.0,"Action,Comedy"
9996,tt6793580,movie,Champions,Campeones,0,2018,\N,124.0,"Comedy,Drama,Family"


The `runtimeMinutes` column isn't of numeric type, because it contains `\N` values that can't be interpreted as numbers. So we need to somehow update those values and make a numeric column out of it.
1. Replace all rows where `runtimeMinutes` contains the imdb encoding for a missing value (`\N`) with `pd.NA` (the pandas type which represents a missing/null value). Note that you might run into some syntax issues. Try to find out how to overcome those.
2. Try to cast the column to a numeric type like this: `.astype(int)`. Why doesn't it work?
3. Try to find alternative ways to make the casting work.
4. Filter the movies that are longer than 2 hours

## Exercise 2
Count how many movies there are per genre. Notice that the `genres` column contains the information about the genre of a movie. But a movie can be assigned to multiple genres. We want to count the movie for every genre.

For example, if we have two movies `sad movie with genres: drama, action` and `dramedy with genres: drama, comedy` then the result counts should be `action: 1, comedy: 1, drama: 2`.

However, the `genres` column is not in a format that makes this counting easy (it's simply a string column). You'll need to transform the `genres` column in a way to simplify the counting. Part of this transformation will be to use the [explode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) function.   

In [18]:
movies['genre'] = movies['genres'].str.split(',')
movies_flat_genre = movies.explode('genre')

movies_flat_genre['genre'].value_counts()
display(movies_flat_genre)

genre
Drama          5591
Comedy         3606
Action         2522
Crime          2071
Adventure      1713
Romance        1670
Thriller       1664
Horror         1241
Mystery        1078
Fantasy         732
Biography       711
Sci-Fi          701
Animation       489
Family          470
History         347
Music           305
War             225
Sport           212
Documentary     212
Musical         128
Western         104
Film-Noir        54
News              4
Name: count, dtype: int64

## Exercise 3
Find out some information about the movie ratings. Specifically, answer following questions
1. What's the movie with the highest average rating?
2. What's the movie with the lowest average rating among the movies with at least 1 million votes (if there are multiple with the same rating, return all of them)
3. Look at the last 10 years and count the number of movies per year as well as the average rating per year

In order to answer the questions, you need to combine the `ratings` dataset with the movies dataset.

In [ ]:
ratings = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep='\t', header=0)

In [ ]:
#display(ratings.describe(), ratings.head(), ratings.info())

In [ ]:
movies_rating_ij = pd.merge(movies, ratings,on=['tconst'])
movies_rating_leftj = pd.merge(movies, ratings,on=['tconst'],how='left') #both looks similar, but left is better in this case, cause movies DF is more important

In [ ]:
#display(movies_rating_ij, movies_rating_leftj)

In [ ]:
movies_rating_leftj.sort_values(by='averageRating', ascending=False).head(1)
# movies_rating_leftj['averageRating'].max()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142.0,Drama,9.3,2802850


In [ ]:
pop_movies = movies_rating_leftj[movies_rating_leftj['numVotes'] >= 1000000]
low_avg_rating = pop_movies['averageRating'].min()
low_rated_pop_movies = pop_movies[pop_movies['averageRating'] == low_avg_rating]
display(pop_movies, low_avg_rating, low_rated_pop_movies )


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142.0,Drama,9.3,2802850
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152.0,"Action, Crime, Drama",9.0,2784495
2,tt1375666,movie,Inception,Inception,0,2010,\N,148.0,"Action, Adventure, Sci-Fi",8.8,2471687
3,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139.0,Drama,8.8,2237022
4,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142.0,"Drama, Romance",8.8,2181499
...,...,...,...,...,...,...,...,...,...,...,...
64,tt2267998,movie,Gone Girl,Gone Girl,0,2014,\N,149.0,"Drama, Mystery, Thriller",8.1,1031144
65,tt0167404,movie,The Sixth Sense,The Sixth Sense,0,1999,\N,107.0,"Drama, Mystery, Thriller",8.2,1025333
66,tt0119217,movie,Good Will Hunting,Good Will Hunting,0,1997,\N,126.0,"Drama, Romance",8.3,1024188
67,tt0477348,movie,No Country for Old Men,No Country for Old Men,0,2007,\N,122.0,"Crime, Drama, Thriller",8.2,1024165


7.9

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
28,tt0499549,movie,Avatar,Avatar,0,2009,\N,162.0,"Action, Adventure, Fantasy",7.9,1361545
34,tt0120338,movie,Titanic,Titanic,0,1997,\N,194.0,"Drama, Romance",7.9,1247901
50,tt0371746,movie,Iron Man,Iron Man,0,2008,\N,126.0,"Action, Adventure, Sci-Fi",7.9,1098807


In [ ]:
import datetime
last_10_years = movies_rating_leftj[(movies_rating_leftj['startYear'] >= datetime.datetime.now().year - 10)]
stats = last_10_years.groupby('startYear').agg({'tconst': 'count', 'averageRating': 'mean'})
display(stats)

,tconst,averageRating
startYear,,
2013,331,6.517825
2014,358,6.506983
2015,331,6.523263
2016,347,6.591066
2017,333,6.430030
2018,355,6.480563
2019,335,6.570448
2020,238,6.184874
2021,287,6.418118
